# Tutorial 5: Solving the Model

In Tutorial 4, we went over how the model is generated when GenX is run using `Run.jl`. In the function `run_genx_case_simple` (or multistage), after `generate_model` is called, `solve_model` is called to solve the EP.

<img src="./files/runcase.png" style="width: 805px; height: auto" align="left">

In this tutorial, we go over how to use JuMP to solve a model, what it looks like to solve GenX, and how to handle infeasibilities.

## Table of Contents
* [A Simple Example](#Simple)
* [GenX](#GenX)
* [Infeasibility](#Infeasibility)

## A Simple Example <a id="Simple"></a>

From Tutorial 4, we have the model:

\begin{align}
& \min 10 x + 15 y &\text{Objective function (cost)}\\ 
& \text{s.t.} & \\
& x + y \geq 10 &\text{Grid Demand}\\
& 55x + 70y \leq \ 1000 &\text{Construction constraint}\\
& 40 x + 5 y \leq 200 &\text{Emissions constraint} \\
& x, y \geq 0 &\text{Non-negativity constraints}\\
\end{align}




In [1]:
using JuMP
using HiGHS

In [2]:
power = Model(HiGHS.Optimizer)

@variable(power,x,Int) # Coal
@variable(power,y,Int) # Wind

@constraint(power, non_neg_x, x >= 0) # Non-negativity constraint (can't have negative power plants!)
@constraint(power, non_neg_y, y >= 0) # Non-negativity constraint

@constraint(power, emissions, 40x + 5y <= 200) # Emisisons constraint
@constraint(power, construction_costs, 55x + 70y <= 1000) # Cost of constructing a new plant

@constraint(power, demand, x + y >= 10) # Grid demand

@expression(power,objective,10x+15y)

@objective(power, Min, objective)

10 x + 15 y

JuMP uses the function `optimize!(model)` to solve the LP:

In [3]:
optimize!(power)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3 rows, 2 cols, 6 nonzeros
3 rows, 2 cols, 6 nonzeros
Objective function is integral with scale 0.2

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   90              inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      130
  Dual bound        130
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    130 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (tota

We can use the function `value.()` to get the value of each variable, and `objective_value()` to get the total objective value.

In [4]:
println("# Coal Plants: ", value.(x))
println("# Wind Farms: ", value.(y))
println("Cost: ", objective_value(power))

# Coal Plants: 4.0
# Wind Farms: 6.0
Cost: 130.0


We can also use the JuMP function `solution_summary` to see more details of the <a href="https://jump.dev/JuMP.jl/stable/manual/solutions/" target="_blank">solution</a>:

In [5]:
solution_summary(power)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 1.30000e+02
  Objective bound    : 1.30000e+02
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 5.69000e-04
  Simplex iterations : 1
  Barrier iterations : -1
  Node count         : 1


## GenX <a id="GenX"></a>

Let's optimize the GenX model created in the last Tutorial. To do so, we'll create the inputs for `generate_model` and run it. 

In [6]:
using GenX

In [7]:
case = joinpath("example_systems/1_three_zones") 

genx_settings = GenX.get_settings_path(case, "genx_settings.yml");
writeoutput_settings = GenX.get_settings_path(case, "output_settings.yml")
setup = GenX.configure_settings(genx_settings,writeoutput_settings)

settings_path = GenX.get_settings_path(case)

### Create TDR_Results
TDRpath = joinpath(case, setup["TimeDomainReductionFolder"])
system_path = joinpath(case, setup["SystemFolder"])

if setup["TimeDomainReduction"] == 1
    GenX.prevent_doubled_timedomainreduction(system_path)
    if !GenX.time_domain_reduced_files_exist(TDRpath)
        println("Clustering Time Series Data (Grouped)...")
        GenX.cluster_inputs(case, settings_path, setup)
    else
        println("Time Series Data Already Clustered.")
    end
end

OPTIMIZER =  GenX.configure_solver(settings_path,HiGHS.Optimizer);

inputs = GenX.load_inputs(setup, case)

Configuring Settings
Time Series Data Already Clustered.
Reading Input CSV Files
Network.csv Successfully Read!
Demand (load) data Successfully Read!
Fuels_data.csv Successfully Read!


[ Info: Thermal.csv Successfully Read.
[ Info: Vre.csv Successfully Read.
[ Info: Storage.csv Successfully Read.
[ Info: Resource_energy_share_requirement.csv Successfully Read.
[ Info: Resource_capacity_reserve_margin.csv Successfully Read.
[ Info: Resource_minimum_capacity_requirement.csv Successfully Read.



Summary of resources loaded into the model:
-------------------------------------------------------
	Resource type 		Number of resources
	Thermal        		3
	VRE            		4
	Storage        		3
Total number of resources: 10
-------------------------------------------------------
Generators_variability.csv Successfully Read!
Validating time basis
Minimum_capacity_requirement.csv Successfully Read!
CO2_cap.csv Successfully Read!
CSV Files Successfully Read In From example_systems/1_three_zones


Dict{Any, Any} with 73 entries:
  "Z"                         => 3
  "LOSS_LINES"                => [1, 2]
  "STOR_HYDRO_SHORT_DURATION" => Int64[]
  "RET_CAP_CHARGE"            => Set{Int64}()
  "pC_D_Curtail"              => [50.0, 45.0, 27.5, 10.0]
  "pTrans_Max_Possible"       => [5.9, 4.0]
  "pNet_Map"                  => [1.0 -1.0 0.0; 1.0 0.0 -1.0]
  "omega"                     => [4.01099, 4.01099, 4.01099, 4.01099, 4.01099, …
  "pMax_Line_Reinforcement"   => [2.95, 2.0]
  "RET_CAP_ENERGY"            => Int64[]
  "RESOURCES"                 => AbstractResource[…
  "COMMIT"                    => [1, 2, 3]
  "pMax_D_Curtail"            => [1.0, 0.04, 0.024, 0.003]
  "STOR_ALL"                  => [8, 9, 10]
  "THERM_ALL"                 => [1, 2, 3]
  "dfCO2CapZones"             => [1 0 0; 0 1 0; 0 0 1]
  "REP_PERIOD"                => 11
  "MinCapReq"                 => [5.0, 10.0, 6.0]
  "PWFU_Num_Segments"         => 0
  "STOR_LONG_DURATION"        => Int64[]
  "THERM_COMMIT_P

In [8]:
EP = GenX.generate_model(setup,inputs,OPTIMIZER)

Discharge Module
Non-served Energy Module
Investment Discharge Module
Unit Commitment Module
Fuel Module
CO2 Module
Investment Transmission Module
Transmission Module
Dispatchable Resources Module
Storage Resources Module
Storage Investment Module
Storage Core Resources Module
Storage Resources with Symmetric Charge/Discharge Capacity Module
Thermal (Unit Commitment) Resources Module
CO2 Policies Module
Minimum Capacity Requirement Module


A JuMP Model
Minimization problem with:
Variables: 120139
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 35112 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 20334 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 97952 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 116439 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: FuelCalculationCommit_single, cCO2Emissions_systemwide, cFuelCalculation_single, cMaxCap, cMaxCapEnergy, cMaxCapEnergyDuration, cMaxFlow_in, cMaxFlow_out, cMaxLineReinforcement, cMaxNSE, cMaxRetCommit, cMaxRetEnergy, cMaxRetNoCommit, cMinCap, cMinCapEnergy, cMinCapEnergyDuration, cNSEPerSeg, cPowerBalance, cSoCBalInterior, cSoCBalStart, cStartFuel_single, cTAuxLimit, cTAuxSum, cTLoss, cZoneMinCapReq, eAvail_Trans_Cap, eCFix, eCFixEnergy, e

The function `solve_model(model, setup)` uses the JuMP function `optimize` to optimize the model:

In [9]:
solution = optimize!(EP) # GenX.solve_model(EP,setup)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
118138 rows, 81183 cols, 467127 nonzeros
110719 rows, 73764 cols, 468669 nonzeros
Presolve : Reductions: rows 110719(-42679); columns 73764(-46375); elements 468669(-46801)
Solving the presolved LP
IPX model has 110719 rows, 73764 columns and 468669 nonzeros
Input
    Number of variables:                                73764
    Number of free variables:                           3696
    Number of constraints:                              110719
    Number of equality constraints:                     16605
    Number of matrix entries:                           468669
    Matrix range:                                       [4e-07, 1e+01]
    RHS range:                                          [8e-01, 4e+03]
    Objective range:                                    [1e-04, 5e+02]
    Bounds range:                                       [2e-03, 1e+01]
Preprocessing
    Dualized model:                    

We can use the command `objective_value` to get the optimial cost of our system as determined by the model. In GenX, the objective value is in dollars.

In [10]:
objective_value(EP)

9998.704689371445

## Infeasibility <a id="Infeasibility"></a>

In some cases, your model may not be able to return a value. This happens when no value can be found that satisfies all constraints. To see this, let's go back to our simple example and add a new constraint to break the model.

\begin{align}
& \min 10 x + 15 y &\text{Objective function (cost)}\\ 
& \text{s.t.} & \\
& x + y \geq 10 &\text{Grid Demand}\\
& 55x + 70y \leq \ 1000 &\text{Construction constraint}\\
& 40 x + 5 y \leq 200 &\text{Emissions constraint} \\
& 7 x + 30 y \geq 500 &\textbf{New Constraint} \\
& x, y \geq 0 &\text{Non-negativity constraints}\\
\end{align}

In [11]:
@constraint(power, new, 7x + 30y >= 500)

new : 7 x + 30 y ≥ 500

In [12]:
print(power)

In [13]:
optimize!(power)

Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In this case, the infeasibility was detected on the presovle since it's clear no solution would fit within all constraints. For information on how to debug an infeasible solution, see the <a href="https://jump.dev/JuMP.jl/stable/manual/solutions/#Conflicts" target="_blank">JuMP documentaion</a>. Some solvers, such as Gurobi, will compute what is causing the conflict, e.g. which constraints are infeasible with one another (HiGHS does not do this). 

GenX version 0.4 has the feature `ComputeConflict` in settings. If the model does not work, try setting `ComputeConflict = 1`, and the conflicting constraints will be returned.

Tutorial 6 describes the solver settings, how to change them, and the effects of PreSolve, Crossover, and Feasibility Tolerance.